# Module 3: 바이트코드 심층 분석 (Bytecode Deep Dive)

이 노트북에서는 Python 코드가 실제로 어떻게 실행되는지 바이트코드 수준에서 살펴 봅니다.

## 목차
1. [바이트코드란?](#1-바이트코드란)
2. [dis 모듈 기본 사용법](#2-dis-모듈-기본-사용법)
3. [코드 객체(__code__) 탐색](#3-코드-객체-탐색)
4. [실습 1: 간단한 연산 바이트코드](#4-실습-1-간단한-연산-바이트코드)
5. [실습 2: 조걸문 바이트코드](#5-실습-2-조걸문-바이트코드)
6. [실습 3: 반복문 바이트코드](#6-실습-3-반복문-바이트코드)
7. [실습 4: 함수 호출 바이트코드](#7-실습-4-함수-호출-바이트코드)
8. [실습 5: bytecode_tracer 도구 사용](#8-실습-5-bytecode_tracer-도구-사용)
9. [실습 6: CFG 시각화](#9-실습-6-cfg-시각화)
10. [연습 문제](#10-연습-문제)

## 1. 바이트코드란?

Python은 인터프리터 언어이지만, 소스 코드를 직접 실행하지 않습니다. 대신 다음과 같은 과정을 거칩니다:

```
소스코드 (.py) → 파싱 → AST (추상 구문 트리) → 컴파일 → 바이트코드 (.pyc) → 실행
```

**바이트코드**는 Python 가상 머신(PVM)이 이해할 수 있는 중간 표현입니다.
- 플랫폼 독립적 (어떤 OS에서도 동일)
- 스택 기반 가상 머신용 명령어 집합
- `.pyc` 파일에 저장되어 재사용됨

각 바이트코드 명령어는 **opcode(명령어 코드)**와 **operand(피연산자)**로 구성됩니다.

In [ ]:
# 필요한 모듈 임포트
import dis
import sys

print(f"Python 버전: {sys.version}")
print(f"\n바이트코드는 Python {sys.version_info.major}.{sys.version_info.minor} 기준으로 작성되었습니다.")

## 2. dis 모듈 기본 사용법

`dis` 모듈은 Python의 디스어셈블러로, 바이트코드를 사람이 읽을 수 있는 형태로 변환합니다.

### 주요 함수
- `dis.dis()`: 함수나 코드 객체의 바이트코드를 출력
- `dis.get_instructions()`: 상세한 명령어 정보를 반환 (이터레이터)
- `dis.stack_effect()`: 특정 명령어의 스택 효과 확인

In [ ]:
# 간단한 함수의 바이트코드 확인
def add(a, b):
    return a + b

print("=== dis.dis() 출력 ===")
dis.dis(add)

### 출력 해석

```
  2           0 RESUME                   0

  3           2 LOAD_FAST                0 (a)
              4 LOAD_FAST                1 (b)
              6 BINARY_OP                0 (+)
             10 RETURN_VALUE
```

- **2, 3**: 소스 코드의 줄 번호
- **0, 2, 4, 6, 10**: 바이트 오프셋 (명령어 위치)
- **RESUME, LOAD_FAST, BINARY_OP, RETURN_VALUE**: opcode (명령어)
- **0, 0, 1, 0**: operand (피연산자 인덱스)
- **(a), (b), (+)**: 피연산자의 실제 값/의미

In [ ]:
# dis.get_instructions() 사용 - 더 상세한 정보
print("=== dis.get_instructions() 출력 ===")
for instr in dis.get_instructions(add):
    print(f"Offset: {instr.offset:3}, Opname: {instr.opname:20}, Arg: {instr.arg}, Argrepr: {instr.argrepr}")

## 3. 코드 객체 탐색

모든 Python 함수는 `__code__` 속성을 가지고 있으며, 이는 **코드 객체**입니다.
코드 객체에는 바이트코드와 관련된 모든 메타데이터가 포함되어 있습니다.

### 주요 속성
- `co_code`: 바이트코드 바이트열 (raw bytecode)
- `co_consts`: 상수 테이블 (리터럴 값들)
- `co_varnames`: 지역변수 이름들
- `co_names`: 전역/빌트인 이름들
- `co_stacksize`: 필요한 스택 크기
- `co_nlocals`: 지역변수 개수
- `co_filename`: 소스 파일명
- `co_name`: 함수/코드 이름

In [ ]:
# 코드 객체 속성 탐색
def example(x, y):
    z = x + y
    return z * 2

code = example.__code__

print("=== 코드 객체 속성 ===")
print(f"co_name: {code.co_name}")
print(f"co_filename: {code.co_filename}")
print(f"co_varnames: {code.co_varnames}")
print(f"co_consts: {code.co_consts}")
print(f"co_names: {code.co_names}")
print(f"co_nlocals: {code.co_nlocals}")
print(f"co_stacksize: {code.co_stacksize}")
print(f"co_code (raw bytes): {code.co_code}")
print(f"co_code length: {len(code.co_code)} bytes")

In [ ]:
# 바이트코드를 16진수로 보기
print("=== Raw Bytecode (Hex) ===")
for i, byte in enumerate(code.co_code):
    if i % 2 == 0:
        print(f"\nOffset {i:3}: ", end="")
    print(f"{byte:02x} ", end="")

## 4. 실습 1: 간단한 연산 바이트코드

`x = 1 + 2` 같은 간단한 연산이 어떤 바이트코드로 변환되는지 분석해 봅시다.

### 핵심 명령어
- `LOAD_CONST`: 상수를 스택에 푸시
- `BINARY_OP`: 이항 연산 수행
- `STORE_FAST`: 스택에서 값을 꺼내 지역변수에 저장

In [ ]:
# 간단한 연산의 바이트코드
def simple_math():
    x = 1 + 2
    return x

print("=== x = 1 + 2 의 바이트코드 ===")
dis.dis(simple_math)

print("\n=== 상수 테이블 ===")
print(f"co_consts: {simple_math.__code__.co_consts}")

### 실행 흐름 시뮬레이션

```
초기: 스택 = []

LOAD_CONST    1 (1)  → 스택 = [1]
LOAD_CONST    2 (2)  → 스택 = [1, 2]
BINARY_OP     0 (+)  → 스택 = [3]  (1과 2를 팝하고 3을 푸시)
STORE_FAST    0 (x)  → 스택 = []   (3을 팝하여 x에 저장)
LOAD_FAST     0 (x)  → 스택 = [3]  (x의 값을 푸시)
RETURN_VALUE        → 스택 = []   (3을 반환)
```

In [ ]:
# 더 복잡한 연산
def complex_math(a, b, c):
    result = (a + b) * c - 10
    return result

print("=== (a + b) * c - 10 의 바이트코드 ===")
dis.dis(complex_math)

## 5. 실습 2: 조걸문 바이트코드

if-else 문은 어떻게 바이트코드로 변환될까요?

### 핵심 명령어
- `POP_JUMP_IF_FALSE`: 조건이 False면 지정된 위치로 점프
- `POP_JUMP_IF_TRUE`: 조건이 True면 지정된 위치로 점프
- `JUMP_FORWARD`: 무조건 앞으로 점프

In [ ]:
# if-else 문의 바이트코드
def check_positive(x):
    if x > 0:
        result = "positive"
    else:
        result = "non-positive"
    return result

print("=== if-else 문의 바이트코드 ===")
dis.dis(check_positive)

### 실행 흐름 분석

```
Offset 0-4: x > 0 비교 수행 → 결과(True/False)를 스택에 푸시
Offset 6: POP_JUMP_IF_FALSE 20
          - 스택에서 값을 팝
          - False이면 offset 20으로 점프 (else 블록)
          - True이면 다음 명령어 실행 (if 블록)
Offset 20-22: else 블록 실행 후 JUMP_FORWARD 6 (if 블록 끝으로)
```

**중요**: Python의 조걸문은 short-circuit 방식으로 동작하며, 조건 평가 후
적절한 블록으로 점프합니다.

In [ ]:
# elif 체인의 바이트코드
def grade(score):
    if score >= 90:
        return 'A'
    elif score >= 80:
        return 'B'
    elif score >= 70:
        return 'C'
    else:
        return 'F'

print("=== elif 체인의 바이트코드 ===")
dis.dis(grade)

## 6. 실습 3: 반복문 바이트코드

for 루프와 while 루프의 바이트코드를 분석합니다.

### 핵심 명령어
- `GET_ITER`: 이터레이터 획득
- `FOR_ITER`: 다음 값을 가져오거나 루프 종료
- `JUMP_BACKWARD`: 루프 시작으로 돌아가기

In [ ]:
# for 루프의 바이트코드
def sum_list(numbers):
    total = 0
    for n in numbers:
        total += n
    return total

print("=== for 루프의 바이트코드 ===")
dis.dis(sum_list)

### for 루프 실행 흐름

```
GET_ITER:    numbers에서 이터레이터 획득 → 스택에 푸시
FOR_ITER 16: 이터레이터에서 다음 값 획득
             - 값이 있으면: 스택에 푸시하고 다음 명령어 실행
             - 값이 없으면: offset + 16으로 점프 (루프 종료)
...          루프 본문 실행
JUMP_BACKWARD 22: FOR_ITER로 돌아가기
```

In [ ]:
# while 루프의 바이트코드
def countdown(n):
    while n > 0:
        n -= 1
    return n

print("=== while 루프의 바이트코드 ===")
dis.dis(countdown)

In [ ]:
# 리스트 컴프리헨션의 바이트코드
def make_squares(n):
    return [x**2 for x in range(n)]

print("=== 리스트 컴프리헨션의 바이트코드 ===")
dis.dis(make_squares)

## 7. 실습 4: 함수 호출 바이트코드

함수가 호출될 때 어떤 바이트코드가 실행되는지 분석합니다.

### 핵심 명령어
- `LOAD_GLOBAL`: 전역 함수/변수 로드
- `LOAD_ATTR`: 객체의 속성 로드
- `CALL`: 함수 호출 (Python 3.11+)
- `CALL_FUNCTION`: 함수 호출 (Python 3.10 이하)

In [ ]:
# 함수 호출의 바이트코드
def greet(name):
    return f"Hello, {name}!"

def call_greet():
    message = greet("World")
    print(message)
    return len(message)

print("=== 함수 호출의 바이트코드 ===")
dis.dis(call_greet)

### 함수 호출 실행 흐름

```
LOAD_GLOBAL greet:  함수 객체를 스택에 푸시
LOAD_CONST "World": 인자를 스택에 푸시
CALL 1:             1개 인자로 함수 호출
                    - 스택에서 인자와 함수를 팝
                    - 결과를 스택에 푸시
```

In [ ]:
# 메서드 호출의 바이트코드
def process_list(items):
    items.append(1)
    result = items.count(1)
    return result

print("=== 메서드 호출의 바이트코드 ===")
dis.dis(process_list)

In [ ]:
# 다중 인자와 키워드 인자 호출
def complex_call():
    # 위치 인자
    range(10)
    # 키워드 인자
    print("hello", end="\n")
    # 여러 인자
    max(1, 2, 3, 4, 5)

print("=== 다양한 함수 호출의 바이트코드 ===")
dis.dis(complex_call)

## 8. 실습 5: bytecode_tracer 도구 사용

이제 우리가 만든 `tools/bytecode_tracer.py` 도구를 사용하여
실제 스택 상태 변화를 추적해 봅시다.

In [ ]:
# 도구 임포트
import sys
sys.path.insert(0, '/home/roach/python-debug')

from tools.bytecode_tracer import trace_execution

# 간단한 함수 추적
def add(a, b):
    return a + b

print("=== 스택 상태 추적: add(1, 2) ===")
trace_execution(add, (1, 2))

In [ ]:
# 지역변수가 있는 함수 추적
def calculate(a, b):
    x = a + b
    y = x * 2
    return y

print("=== 스택 상태 추적: calculate(3, 4) ===")
steps = trace_execution(calculate, (3, 4))

# 각 단계의 지역변수 상태 확인
print("\n=== 각 단계별 지역변수 상태 ===")
for step in steps:
    if step.locals_snapshot:
        print(f"Offset {step.offset}: {step.locals_snapshot}")

In [ ]:
# 복잡한 함수 추적
def factorial(n):
    if n <= 1:
        return 1
    return n * factorial(n - 1)

print("=== 스택 상태 추적: factorial(3) ===")
# 참고: 재귀 호출은 추적되지 않음 (한 함수의 바이트코드만 추적)
trace_execution(factorial, (3,))

In [ ]:
# 리스트 연산 추적
def list_operations():
    items = [1, 2, 3]
    items.append(4)
    return len(items)

print("=== 스택 상태 추적: list_operations() ===")
trace_execution(list_operations)

## 9. 실습 6: CFG 시각화

`tools/cfg_visualizer.py`를 사용하여 함수의 **제어 흐름 그래프(CFG)**를
시각화해 봅시다.

**참고**: 이 섹션은 `graphviz`가 설치되어 있어야 실행됩니다.
설치되지 않은 경우 시각화가 생략됩니다.

In [ ]:
# graphviz 설치 확인
try:
    import graphviz
    HAS_GRAPHVIZ = True
    print("✓ graphviz가 설치되어 있습니다.")
except ImportError:
    HAS_GRAPHVIZ = False
    print("✗ graphviz가 설치되지 않았습니다.")
    print("  pip install graphviz로 설치할 수 있습니다.")

In [ ]:
if HAS_GRAPHVIZ:
    from tools.cfg_visualizer import visualize_cfg
    
    # if 문의 CFG
    def test_if(x):
        if x > 0:
            return x
        else:
            return -x
    
    print("=== if 문의 CFG 생성 ===")
    output_path = visualize_cfg(test_if, 'outputs/cfg/test_if.png')
    print(f"CFG 저장됨: {output_path}")
else:
    print("graphviz가 없어 CFG 생성을 건너뜁니다.")

In [ ]:
if HAS_GRAPHVIZ:
    # for 루프의 CFG
    def test_loop(n):
        total = 0
        for i in range(n):
            total += i
        return total
    
    print("=== for 루프의 CFG 생성 ===")
    output_path = visualize_cfg(test_loop, 'outputs/cfg/test_loop.png')
    print(f"CFG 저장됨: {output_path}")
else:
    print("graphviz가 없어 CFG 생성을 건너뜁니다.")

In [ ]:
if HAS_GRAPHVIZ:
    # while 루프의 CFG
    def test_while(n):
        i = 0
        while i < n:
            i += 1
        return i
    
    print("=== while 루프의 CFG 생성 ===")
    output_path = visualize_cfg(test_while, 'outputs/cfg/test_while.png')
    print(f"CFG 저장됨: {output_path}")
else:
    print("graphviz가 없어 CFG 생성을 건너뜁니다.")

In [ ]:
if HAS_GRAPHVIZ:
    # 중첩 조건문의 CFG
    def complex_condition(x, y):
        if x > 0:
            if y > 0:
                return "both positive"
            else:
                return "x positive only"
        else:
            return "x non-positive"
    
    print("=== 중첩 조건문의 CFG 생성 ===")
    output_path = visualize_cfg(complex_condition, 'outputs/cfg/complex_condition.png')
    print(f"CFG 저장됨: {output_path}")
else:
    print("graphviz가 없어 CFG 생성을 건너뜁니다.")

### CFG 색상 의미

생성된 CFG에서 블록 색상의 의미:
- **파란색 (#E3F2FD)**: 시작 블록 (Entry)
- **녹색 (#C8E6C9)**: 종료 블록 (Return)
- **주황색 (#FFE0B2)**: 조건부 블록 (if 문)
- **노란색 (#FFF9C4)**: 반복 블록 (for/while)
- **회색 (#F5F5F5)**: 일반 블록

### 엣지 스타일
- **실선**: 무조건 실행 (fall-through)
- **점선**: 조건부 실행 (jump)
- **True/False 레이블**: 조건 분기 방향

## 10. 연습 문제

다음 문제들을 직접 풀어보며 바이트코드 분석 능력을 키워보세요.

### 문제 1: 바이트코드 예측

다음 함수의 바이트코드를 예측해보고, 실제로 `dis.dis()`로 확인해 보세요.

In [ ]:
# 문제 1: 이 함수의 바이트코드를 예측해보세요
def mystery1(x):
    return x * 2 + 1

# 여기에 예측을 작성해보세요:
# 1. x를 스택에 로드: LOAD_FAST
# 2. ...

print("=== 정답 확인 ===")
dis.dis(mystery1)

### 문제 2: 조건문 분석

다음 코드에서 `and`와 `or`가 어떻게 바이트코드로 변환되는지 분석하세요.

In [ ]:
# 문제 2: 논리 연산자의 바이트코드
def check_logic(a, b, c):
    if a > 0 and b > 0 or c > 0:
        return True
    return False

print("=== 논리 연산자의 바이트코드 ===")
dis.dis(check_logic)

### 문제 3: 루프 최적화

다음 두 함수의 바이트코드를 비교하고, 어떤 차이가 있는지 분석하세요.

In [ ]:
# 문제 3: 두 루프 구현 비교
def loop_with_range(n):
    total = 0
    for i in range(n):
        total += i
    return total

def loop_with_while(n):
    total = 0
    i = 0
    while i < n:
        total += i
        i += 1
    return total

print("=== for + range ===")
dis.dis(loop_with_range)

print("\n=== while ===")
dis.dis(loop_with_while)

### 문제 4: 스택 추적

다음 함수를 `trace_execution`으로 추적하고, 각 단계의 스택 상태를 분석하세요.

In [ ]:
# 문제 4: 스택 추적 연습
def stack_practice(a, b, c):
    x = a + b
    y = x * c
    z = y - a
    return z

print("=== 스택 추적 ===")
trace_execution(stack_practice, (1, 2, 3))

print("\n=== 바이트코드 ===")
dis.dis(stack_practice)

### 문제 5: CFG 생성

다음 함수의 CFG를 생성하고, 제어 흐름을 분석하세요.

In [ ]:
# 문제 5: CFG 분석
def find_max(a, b, c):
    if a > b:
        if a > c:
            return a
        else:
            return c
    else:
        if b > c:
            return b
        else:
            return c

if HAS_GRAPHVIZ:
    print("=== find_max CFG 생성 ===")
    output_path = visualize_cfg(find_max, 'outputs/cfg/find_max.png')
    print(f"CFG 저장됨: {output_path}")
    
    print("\n=== 바이트코드 ===")
    dis.dis(find_max)
else:
    print("graphviz가 없어 바이트코드만 출력합니다.")
    dis.dis(find_max)

## 정리

이 노트북에서 학습한 내용:

1. **바이트코드 기초**: Python 코드가 어떻게 바이트코드로 변환되는지
2. **dis 모듈**: `dis.dis()`와 `dis.get_instructions()` 사용법
3. **코드 객체**: `__code__` 속성과 주요 메타데이터
4. **명령어 분석**: LOAD, STORE, BINARY_OP, CALL 등 핵심 명령어
5. **제어 흐름**: 조건문과 반복문의 점프 명령어
6. **스택 추적**: `bytecode_tracer`로 스택 상태 변화 확인
7. **CFG 시각화**: `cfg_visualizer`로 제어 흐름 그래프 생성

### 다음 단계
- 더 복잡한 예제 (예외 처리, 제너레이터 등) 분석
- Python 버전별 바이트코드 차이 비교
- 직접 바이트코드를 생성/수정해 보기